## **Summary of `prediction.ipynb`**

1. **Loading the Trained Model**:
   - A pre-trained deep learning model (`my_model.keras`) is loaded using TensorFlow's `load_model` function.
   - The optimizer's state is skipped during loading by setting `compile=False`.

2. **Loading Preprocessing Objects**:
   - Preprocessing objects such as a scaler (`scaler.pkl`), a label encoder for gender (`label_encoder_gender.pkl`), and a one-hot encoder for geography (`ohe_geography.pkl`) are loaded using Python's `pickle` module.
   - These objects were previously saved during the training phase and are used to preprocess input data for predictions.

3. **Input Data Preparation**:
   - Example input data is defined in a dictionary format, containing features such as `CreditScore`, `Geography`, `Gender`, `Age`, `Balance`, etc.
   - The `Geography` field is one-hot encoded using the loaded one-hot encoder (`one_hot_encoder_geo`), ensuring compatibility with the model's input format.

4. **Preprocessing Steps**:
   - The `Geography` field is transformed into one-hot encoded features using the `OneHotEncoder`.
   - The input data is scaled using the loaded scaler to match the format used during training.

5. **Prediction**:
   - The preprocessed input data is passed to the loaded model for prediction.
   - The model outputs the prediction, which can be interpreted as the likelihood of a specific outcome (e.g., customer churn).

---

### **Purpose of these codes**
- Load a pre-trained artificial neural network (ANN) model for classification tasks.
- Preprocess new input data using saved preprocessing objects (scaler, encoders).
- Make predictions on the input data and interpret the results.

## Predictions

In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from tensorflow.keras.models import load_model
import pickle

In [23]:
## Load the trained model, pickle file
# Disable Loading the Optimizer State:When loading the model, you can skip loading the optimizer's state by specifying compile=False:
model = load_model('my_model.keras', compile=False)

# load scaler label and one hot encoder are saved in pickle file
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('ohe_geography.pkl', 'rb') as file:
    one_hot_encoder_geo = pickle.load(file)

In [24]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [25]:
# Convert input data to a DataFrame with the correct column name
input_df = pd.DataFrame(input_data, index=[0])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [26]:
# One-hot encode 'Geography'
geo_encoded = one_hot_encoder_geo.transform(input_df[['Geography']])
geo_encoded_df = pd.DataFrame(geo_encoded, columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))

geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [27]:
# Encode Categorical - Label Encoding
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [28]:
#combine the one-hot encoded columns with the original input DataFrame
input_df = pd.concat([input_df, geo_encoded_df], axis=1)
input_df.drop('Geography', axis=1, inplace=True)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [29]:
#Scale the input data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [30]:
#lets predict if person is going to leave the bank or not - the customer churn
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step


array([[0.04428104]], dtype=float32)

In [31]:
# Convert the prediction to a human-readable format
if prediction > 0.5:
    print('The customer is likely to leave the bank.')
else:
    print('The customer is likely to stay with the bank.')

The customer is likely to stay with the bank.
